# Piperack Generator 1.0

## Module imports and connection to STAAD

In [ ]:
from pyperclip import copy
from IPython.display import display, Markdown
from base.helper.general import *
from base.geometry_base.line import *
from base.geometry_base.rectangle import *
from base.staad_base.geometry import *
from base.structural_elements.beam import *
from base.structural_elements.column import *
from base.structural_elements.brace import *
from base.piperack.portal import *
from base.piperack.piperack import *
from base.piperack.tier import *
from base.load.uniform_load import *
from base.staad_base.design import *
from base.staad_base.property import *
from output.md_output import *

from base.staad_base.optimise_member import *

openSTAAD,STAAD_objects = get_openSTAAD()

geometry = STAAD_objects.geometry
property = STAAD_objects.property
output = STAAD_objects.output
support = STAAD_objects.support
load = STAAD_objects.load

add_beams = add_beams_fn(geometry=geometry)
select_beams = select_beams_fn(geometry=geometry)
assign_profile = assign_profile(property=property)
assign_specification = assign_specification(property=property)

## Piperack Inputs

### Portal and Structure

In [ ]:
base_point_of_first_portal : Point3D = Point3D(0,0,0)
width_of_piperack = 8
portal_distances = [0,8,16,24,32,40,48]
column_distances = [0,width_of_piperack]
braces_placement = [False,True,False,False,True,False]
brace_pattern = BracePattern.X_Pattern

#### Tiers

##### Piping Tiers

In [ ]:
operating_load = UniformLoad(load_case=LoadCase.OperatingLoad)
empty_load = UniformLoad(load_case=LoadCase.EmptyLoad)

tg_gx = UniformLoad(load_case=LoadCase.ThermalGravity_GX,direction=MemberDirection.GX)
tg_gz = UniformLoad(load_case=LoadCase.ThermalGravity_GZ,direction=MemberDirection.GZ)
tl_gx = UniformLoad(load_case=LoadCase.ThermalLateral_GX,direction=MemberDirection.GX)
tl_gz = UniformLoad(load_case=LoadCase.ThermalLateral_GZ,direction=MemberDirection.GZ)

clt_gy = UniformLoad(load_case=LoadCase.ContigencyLoadTransverse,direction=MemberDirection.GY)
clt_gz = UniformLoad(load_case=LoadCase.ContigencyLoadTransverse,direction=MemberDirection.GZ)

tiers = [
    Tier(base=base_point_of_first_portal.shift_y(3),loads=[operating_load.set_force_value(-0.4)]).set_intermediate_transverse_beam(False),
    Tier(base=base_point_of_first_portal.shift_y(6),loads=[operating_load.set_force_value(-0.4)]),
    Tier(base=base_point_of_first_portal.shift_y(9.5),loads=[operating_load.set_force_value(-0.35)]),
    Tier(base=base_point_of_first_portal.shift_y(12),loads=[operating_load.set_force_value(-0.3)]),
    Tier(base=base_point_of_first_portal.shift_y(14.5),tier_type= TierType.ElectricalIntrumentation).set_intermediate_transverse_beam(False),
    Tier(base=base_point_of_first_portal.shift_y(17.5),tier_type= TierType.Flare).set_intermediate_transverse_beam(False),
]

tiers = sorted(tiers,key = lambda tier_x : tier_x.base.y)

for tier in tiers:
    if(tier.loads):
        tier_op_load_x = tier.loads[0]
        if(tier_op_load_x):
            tier.add_load(empty_load.set_force_value(tier_op_load_x.force_value*0.4))
            tier.add_load(tg_gx.set_force_value(tier_op_load_x.force_value*(-0.025)))
            tier.add_load(tg_gz.set_force_value(tier_op_load_x.force_value*(-0.125)))
            tier.add_load(tl_gx.set_force_value(tier_op_load_x.force_value*(-0.05)))
            tier.add_load(tl_gz.set_force_value(tier_op_load_x.force_value*(-0.05)))

for tier in tiers:
    if(tier.loads):
        tier_op_load_x = tier.loads[0]
        if(tier_op_load_x):
            tier.add_clt_load(clt_gy.set_force_value(tier_op_load_x.force_value*0.4))
            tier.add_clt_load(clt_gz.set_force_value(tier_op_load_x.force_value*(-1*0.04)))
            
tier_elevations = [tier.base.y for tier in tiers]
tier_dict = { tier.base.y : tier for tier in tiers }

display(Markdown(create_tiers_markdown_table(tiers)))

##### E&I And Flare Details

In [ ]:
flare_lines = [Line3D(Point3D(2.5,17.5,portal_distances[0]),Point3D(2.5,17.5,portal_distances[-1]))]

ww_width = 0.8
ww_1_x = 1
ww_2_x = 4.5
ww_lines = [
            Line3D(Point3D(ww_1_x,14.5,portal_distances[0]),Point3D(ww_1_x,14.5,portal_distances[-1])),
            Line3D(Point3D(ww_1_x+ww_width,14.5,portal_distances[0]),Point3D(ww_1_x+ww_width,14.5,portal_distances[-1])),
            Line3D(Point3D(ww_2_x,14.5,portal_distances[0]),Point3D(ww_2_x,14.5,portal_distances[-1])),
            Line3D(Point3D(ww_2_x+ww_width,14.5,portal_distances[0]),Point3D(ww_2_x+ww_width,14.5,portal_distances[-1]))
        ]

duct_width = 1.2
duct_1_x = 6
duct_lines = [
                Line3D(Point3D(duct_1_x,14.5,portal_distances[0]),Point3D(duct_1_x,14.5,portal_distances[-1])),
                Line3D(Point3D(duct_1_x+duct_width,14.5,portal_distances[0]),Point3D(duct_1_x+duct_width,14.5,portal_distances[-1])),
            ]

tree_x = 3
tree_support = [Line3D(Point3D(tree_x,14.5,portal_distances[0]),Point3D(tree_x,14.5,portal_distances[-1]))]

#### Longitudinal Beam Elevations


In [ ]:
long_beam_elevations = [*[tiers[tier_i].base.mid(tiers[tier_i+1].base).y for tier_i in range(len(tiers)-1)],tier_elevations[-1]]

display(long_beam_elevations)

### Foundation

In [ ]:
pedestal_height = 2
foundation_depth = 1

## Profiles

In [ ]:
concrete_profile_list = {
    PiperackMembers.Pedestals : Rectangle(1.2,0.8)
}

steel_profile_list = {
    PiperackMembers.Columns : 'WPB700X300X240.51',
    PiperackMembers.TierBeams : 'WPB600X300X128.79',
    PiperackMembers.LongitudinalBeams : 'WPB250X250X73.14',
    PiperackMembers.PlanBracing : '100X100X6.0SHS',
    PiperackMembers.VerticalBracing : '150X150X8.0SHS',
    PiperackMembers.Stubs : 'ISMB300',
    PiperackMembers.IntermediateTransverseBeams : 'ISMB300',
    PiperackMembers.IntermediateLongitudinalBeams : 'ISMB300',
    PiperackMembers.FlareSupportMembers : 'WPB700X300X240.51',
    PiperackMembers.TreeSupportMembers : 'ISMC400',
    PiperackMembers.DuctSupportMembers : 'ISMC250',
    PiperackMembers.WWSupportMembers : 'ISMC200'
}

profile_ids = {}

for member_type,profile in concrete_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        profile_ids[member_type] = create_concrete_beam_property(property,profile)

for member_type,profile in steel_profile_list.items():
    if ((isinstance(profile, str) and profile.isdigit()) or isinstance(profile, (int, float))):
        profile_ids[member_type] = int(profile)
    else:
        if('SHS' in profile):
            profile_ids[member_type] = create_steel_beam_property(property,35,profile,0,0,0)
        else:
            profile_ids[member_type] = create_steel_beam_property(property,10,profile,0,0,0)
        
display(Markdown(create_profile_markdown_table(concrete_profile_list, steel_profile_list, profile_ids)))

## Supports

In [ ]:
support_id = support.CreateSupportFixed()

## Specifications

### Member Releases

In [ ]:
start_release_spec = property.CreateMemberReleaseSpec(0,set_DOFReleaseArray())
end_release_spec = property.CreateMemberReleaseSpec(1,set_DOFReleaseArray())
set_start_end_release = get_start_end_release_function(property=property,start_release_spec=start_release_spec,end_release_spec=end_release_spec)

### Truss and Offset

In [ ]:
foundation_depth = 1
truss_spec = property.CreateMemberTrussSpec()
member_offset_spec = property.CreateMemberOffsetSpec(0,0,0,foundation_depth,0)

display(Markdown(create_spec_markdown_table(start_release_spec,end_release_spec,truss_spec,member_offset_spec)))

## Member Creation

### Primary Members

In [ ]:
#portals 
longitudinal_beams = []
stubs = []
intermediate_transverse_beams = []
intermediate_long_beams = []
plan_braces = []
vertical_braces = []
portal_beam_ids = [] 
portal_column_ids = [] 
portal_pedestal_ids = [] 
support_node_ids = []
long_beam_ids = [] 
stub_ids = []
intermediate_transverse_ids = []
intermediate_long_ids = []
plan_brace_ids = []
vertical_brace_ids = []
portal_count = len(portal_distances)

# Create first portal
portal : PiperackPortal = PiperackPortal(base=base_point_of_first_portal) 
for column in column_distances:
    support_point = Point3D(column,0,0)-Point3D(0,pedestal_height,0)
    portal.add_pedestal(Column3D(base=support_point,height=pedestal_height))
    portal.add_column(Column3D(base=Point3D(column,0,0),height=tier_elevations[-1])) 
    
for tier in tier_elevations:
    for column_i in range(len(column_distances)-1):
        portal.add_beam(Beam3D(start=Point3D(column_distances[column_i],tier,0),end=Point3D(column_distances[column_i+1],tier,0))) 

# Create all portals by shifting the first portal
portals = [portal.shift(Point3D(0,0,dist)) for dist in portal_distances]

# Create longitudinal beams between portals
for portal_i in range(len(portals)-1):
    for column in column_distances:
        for long_beam in long_beam_elevations:
            long_beam_x = Beam3D(start=Point3D(column,long_beam,portals[portal_i].base.z),end=Point3D(column,long_beam,portals[portal_i+1].base.z))
            longitudinal_beams.append(long_beam_x)


In [ ]:
# Add portal members to model and collect IDs
for portal in portals:
    portal_beam_ids = [*portal_beam_ids,*add_beams(portal.beams)]
    portal_column_ids = [*portal_column_ids,*add_beams(portal.columns)]
    portal_pedestal_ids = [*portal_pedestal_ids,*add_beams(portal.pedestals)]

# Assign profiles to portal members
portal_beam_assign = assign_profile(portal_beam_ids,profile_ids[PiperackMembers.TierBeams])
portal_column_assign = assign_profile(portal_column_ids,profile_ids[PiperackMembers.Columns])
portal_pedestal_assign = assign_profile(portal_pedestal_ids,profile_ids[PiperackMembers.Pedestals])

# Add longitudinal beams and assign properties
long_beam_ids = add_beams(longitudinal_beams)
long_release = set_start_end_release(long_beam_ids)
long_assign = assign_profile(long_beam_ids,profile_ids[PiperackMembers.LongitudinalBeams])

# Add support nodes
for portal in portals:
    for pedestal in portal.pedestals:
        support_node_ids.append(add_support_node(geometry,support,pedestal.start,support_id))

# Assign specifications
assign_specification(portal_pedestal_ids,member_offset_spec)

# Create comprehensive markdown table
markdown_output = "## Primary Members Created\n\n"
markdown_output += "| Member Types     | IDs                  | Profile Assignment     | Release Assignment     | Notes |\n"
markdown_output += "| ---              | ---                  | ---                    | ---                    | ---   |\n"
markdown_output += f"| Portal Columns   | {portal_column_ids}  | {portal_column_assign} | -                      | Vertical support members |\n"
markdown_output += f"| Portal Pedestals | {portal_pedestal_ids}| {portal_pedestal_assign}| -                     | Foundation connections |\n"
markdown_output += f"| Portal Beams     | {portal_beam_ids}    | {portal_beam_assign}   | -                      | Transverse tier beams |\n"
markdown_output += f"| Long Beams       | {long_beam_ids}      | {long_assign}          | {long_release}         | Longitudinal beams between portals |\n"
markdown_output += f"| Support Nodes    | {support_node_ids}   | -                      | -                      | Foundation support points |\n"

display(Markdown(markdown_output))

### Secondary Members

In [ ]:
# Group longitudinal beams by Y coordinate
long_beams_y = group_beams_by_y(longitudinal_beams)

# Create vertical braces    
if(brace_pattern == BracePattern.V_Pattern):
    previous_y = None
    for y_i,tier_y in long_beams_y.items():
        base = portal.base.y if not previous_y else previous_y
        long_beams_sorted = beams_sorted_yxz(tier_y)

        for beam_i in range(len(long_beams_sorted)-1):
            portal_i = beam_i%(len(portals)-1)
            if(braces_placement[portal_i]):
                
                long_beam_i : Beam3D = long_beams_sorted[beam_i]

                vb_1 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.start.z),
                                end=long_beam_i.mid())
                vb_2 = Beam3D(  start=Point3D(long_beam_i.start.x,base,long_beam_i.end.z),
                                end=long_beam_i.mid())
                    
                vertical_braces.append(vb_1)
                vertical_braces.append(vb_2)

        previous_y = y_i
        
elif(brace_pattern == BracePattern.X_Pattern):
# Initialize variables
    sorted_y = sorted(long_beams_y.keys())  # Sort keys of long_beams_y
    vertical_braces = []  # Ensure vertical_braces is defined (assuming it's a list)

    for portal_i, current_y_key in enumerate(sorted_y):
        tier_y = long_beams_y[current_y_key]
        
        # Determine previous and next y-keys
        previous_y_key = sorted_y[portal_i - 1] if portal_i > 0 else portal.base.y
        next_y_key = sorted_y[portal_i + 1] if portal_i + 1 < len(sorted_y) else None
        
        # Process only even-indexed groups
        if portal_i % 2 == 0:
            base_y = portal.base.y if previous_y_key is None else previous_y_key
            long_beams_sorted = beams_sorted_yxz(tier_y)
            
            # Iterate through beams (excluding the last one)
            for beam_idx in range(len(long_beams_sorted) - 1):
                portal_idx = beam_idx % (len(portals) - 1)
                
                if braces_placement[portal_idx]:
                    current_beam = long_beams_sorted[beam_idx]
                    
                    pt_1 = Point3D(current_beam.start.x, base_y, current_beam.start.z)
                    pt_2 = current_beam.mid()
                    angle_with_hzl_1 = Line3D(pt_1,pt_2).angle_with_xy_plane()
                    
                    # Create vertical braces from base
                    if(30<=angle_with_hzl_1<=60):
                        vb_1 = Beam3D(
                            start=Point3D(current_beam.start.x, base_y, current_beam.start.z),
                            end=current_beam.mid()
                        )
                        vb_2 = Beam3D(
                            start=Point3D(current_beam.start.x, base_y, current_beam.end.z),
                            end=current_beam.mid()
                        )
                        
                        vertical_braces.extend([vb_1, vb_2])
                    
                    else:
                        total_z = abs(pt_4.z-pt_3.z)
                        vb_5 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z+total_z/2),
                            end=current_beam.mid()
                        )
                        vb_6 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z-total_z/2),
                            end=current_beam.mid()
                        )
                        vb_7 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z).shift_y(current_beam.start.y),
                            end=vb_5.start
                        )
                        vb_8 = Beam3D(
                            start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z).shift_y(current_beam.start.y),
                            end=vb_6.start
                        )
                        vertical_braces.extend([vb_5, vb_6 , vb_7 , vb_8])
                        
                    # Add braces to next y-key if it exists
                    if (next_y_key is not None):
                        pt_3 = Point3D(current_beam.start.x, next_y_key, current_beam.start.z)
                        pt_4 = current_beam.mid()
                        angle_with_hzl_2 = Line3D(pt_3,pt_4).angle_with_xy_plane()
                        
                        if(30<=angle_with_hzl_2<=60):
                        
                            vb_3 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z),
                                end=current_beam.mid()
                            )
                            vb_4 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z),
                                end=current_beam.mid()
                            )
                            
                            vertical_braces.extend([vb_3, vb_4])
                        else:
                            total_z = abs(pt_4.z-pt_3.z)
                            vb_5 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z+total_z/2),
                                end=current_beam.mid()
                            )
                            vb_6 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z-total_z/2),
                                end=current_beam.mid()
                            )
                            vb_7 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.start.z).shift_y(current_beam.start.y),
                                end=vb_5.start
                            )
                            vb_8 = Beam3D(
                                start=Point3D(current_beam.start.x, next_y_key, current_beam.end.z).shift_y(current_beam.start.y),
                                end=vb_6.start
                            )
                            vertical_braces.extend([vb_5, vb_6 , vb_7 , vb_8])
                            

# Add vertical braces and assign properties
vertical_brace_ids = add_beams(vertical_braces)
vertical_brace_spec = assign_specification(vertical_brace_ids,truss_spec)
vertical_brace_assign = assign_profile(vertical_brace_ids,profile_ids[PiperackMembers.VerticalBracing])

In [ ]:

stub_braces = []
lower_stubs = []
intermediate_transverse_beams_with_stub = []

# Create stub beams between adjacent longitudinal beams
create_stub = lambda beamA,beamB : Beam3D(start=beamA.mid(),end=beamB.mid())

for long_beam_i in range(len(longitudinal_beams)-1):
    long_ii:Beam3D = longitudinal_beams[long_beam_i]
    long_ij:Beam3D = longitudinal_beams[long_beam_i+1]
    if(long_ii.start.eq_x(long_ij.start) and long_ii.end.eq_z(long_ij.end)):
        stubs.append(create_stub(long_ii,long_ij))

if(len(tiers)>0):
    lowest_tier = tiers[0]
    lowest_tier_y = lowest_tier.base.y
    
    if(lowest_tier.intermediate_transverse_beam):
        lowest_long_beam = sorted(longitudinal_beams,key = lambda beam : beam.start.y)[0]
        
        if lowest_long_beam.start.y in long_beams_y:
            lowest_long_beams = long_beams_y[lowest_long_beam.start.y]

            ht_diff = lowest_long_beam.start.y - lowest_tier_y
            
            for long_beam_i in range(0, len(lowest_long_beams), 2):
                first_beam : Beam3D = lowest_long_beams[long_beam_i]
                second_beam :Beam3D = lowest_long_beams[long_beam_i + 1] if long_beam_i + 1 < len(lowest_long_beams) else None
                
                first_beam_mid = first_beam.mid()
                second_beam_mid = second_beam.mid()
                
                lower_stubs.append(Beam3D(start=first_beam_mid.shift_y(lowest_tier_y),end=first_beam_mid.shift_y(lowest_long_beam.start.y)))
                lower_stubs.append(Beam3D(start=second_beam_mid.shift_y(lowest_tier_y),end=second_beam_mid.shift_y(lowest_long_beam.start.y)))
                                
                stub_braces.append(Beam3D(start=first_beam_mid.shift_y(lowest_tier_y),end=first_beam_mid.shift_y(lowest_long_beam.start.y).shift_z(first_beam_mid.z+ht_diff)))
                stub_braces.append(Beam3D(start=second_beam_mid.shift_y(lowest_tier_y),end=second_beam_mid.shift_y(lowest_long_beam.start.y).shift_z(first_beam_mid.z+ht_diff)))

                intermediate_transverse_beams_with_stub.append(Beam3D(start=first_beam_mid,end=second_beam_mid).shift_to_y(lowest_tier_y))

stubs = beams_sorted_yzx(stubs)

# Add stubs and assign properties
stub_ids = add_beams(stubs)
stub_assign = assign_profile(stub_ids,profile_ids[PiperackMembers.Stubs])
stub_release = set_start_end_release(stub_ids)

lower_stub_ids = add_beams(lower_stubs)
lower_stub_assign = assign_profile(lower_stub_ids,profile_ids[PiperackMembers.Stubs])
lower_stub_release = assign_specification(lower_stub_ids,end_release_spec)

stub_brace_ids = add_beams(stub_braces)
stub_assign = assign_profile(stub_brace_ids,profile_ids[PiperackMembers.VerticalBracing])
stub_brace_spec = assign_specification(stub_brace_ids,truss_spec)

intermediate_transverse_beams_with_stub_ids = add_beams(intermediate_transverse_beams_with_stub)
intermediate_transverse_beams_with_stub_assign = assign_profile(intermediate_transverse_beams_with_stub_ids,profile_ids[PiperackMembers.IntermediateTransverseBeams])

In [ ]:
# Create intermediate transverse beams from stubs
for stub_i in range(0, len(stubs), 2):
    if stub_i + 1 < len(stubs):  
        stub1, stub2 = stubs[stub_i], stubs[stub_i + 1]
        stub1_mid,stub2_mid = stub1.mid(),stub2.mid()
        
        for tier_y,tier in tier_dict.items():
            if( stub1.start.y <= tier_y <= stub1.end.y and tier.intermediate_transverse_beam):
                intermediate_transverse_beams.append(Beam3D(start=stub1_mid,end=stub2_mid)\
                                                        .shift_to_y(tier_y))
                                                        # .shift_to_y(closest_to(tier_elevations,stub1_mid.mid(stub2_mid).y)))
    
# Add intermediate transverse beams and assign properties
intermediate_transverse_ids = add_beams(intermediate_transverse_beams)
inter_trans_assign = assign_profile(intermediate_transverse_ids,profile_ids[PiperackMembers.IntermediateTransverseBeams])
inter_trans_release = set_start_end_release(intermediate_transverse_ids)

# Create intermediate longitudinal beams and plan braces from portal beams
portal_beams_y = group_beams_by_y([beam for portal in portals for beam in portal.beams])
for tier_y,tier in tier_dict.items():
    if(tier_y in portal_beams_y):
        sorted_beams = beams_sorted_yzx(portal_beams_y[tier_y])
        
        for beam_i in range(len(sorted_beams)-1):
            beam1 : Beam3D = sorted_beams[beam_i]
            beam2 : Beam3D = sorted_beams[beam_i+1]

            if(tier.tier_type == TierType.Piping):
                beam1_mid = beam1.mid()
                beam2_mid = beam2.mid()

                if(tier.intermediate_transverse_beam):
                    beam_mid = beam1_mid.mid(beam2_mid)
                    intermediate_long_beams.append(Beam3D(start=beam1_mid,end=beam_mid))
                    intermediate_long_beams.append(Beam3D(start=beam_mid,end=beam2_mid))
                else:
                    intermediate_long_beams.append(Beam3D(start=beam1_mid,end=beam2_mid))

                if(braces_placement[beam_i]):
                    mid_pt = beam1.start.mid(beam2.end)
                    for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                        plan_braces.append(Beam3D(start=pt_x,end=mid_pt))

            elif(tier.tier_type == TierType.Flare): 
                if(braces_placement[beam_i]):
                    flare_level_x = beam1.start.mid(beam2.end)
                    
                    for line in flare_lines:
                        if(line.start.eq_y(tier_y)):
                            flare_level_x = beam1.start.mid(beam2.end).shift_x(line.start.x)

                    mid_pt = flare_level_x
                    for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                        plan_braces.append(Beam3D(start=pt_x,end=mid_pt))

            elif(tier.tier_type == TierType.ElectricalIntrumentation):
                if(braces_placement[beam_i]):
                    tree_level_x = beam1.start.mid(beam2.end)
                    
                    if(tree_support):
                        for line in tree_support:
                            if(line.start.eq_y(tier_y)):
                                tree_level_x = beam1.start.mid(beam2.end).shift_x(line.start.x)
                    elif(not tree_support and duct_lines):
                        for line in duct_lines:
                            if(line.start.eq_y(tier_y)):
                                tree_level_x = beam1.start.mid(beam2.end).shift_x(line.start.x)

                    mid_pt = tree_level_x
                    for pt_x in [beam1.start,beam1.end,beam2.start,beam2.end]:
                        plan_braces.append(Beam3D(start=pt_x,end=mid_pt))


# Add intermediate longitudinal beams and assign properties
intermediate_long_ids = add_beams(intermediate_long_beams)
inter_long_assign = assign_profile(intermediate_long_ids,profile_ids[PiperackMembers.IntermediateLongitudinalBeams])
inter_long_release = set_start_end_release(intermediate_long_ids)

# Add plan braces and assign properties
plan_brace_ids = add_beams(plan_braces)
plan_brace_spec = assign_specification(plan_brace_ids,truss_spec)
plan_brace_assign = assign_profile(plan_brace_ids,profile_ids[PiperackMembers.PlanBracing])

# Create comprehensive markdown table for secondary members
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types              | IDs                        | Profile Assignment      | Release Assignment      | Specification           | Function |\n"
markdown_output += "| ---                       | ---                        | ---                     | ---                     | ---                     | ---      |\n"
markdown_output += f"| Stubs                    | {stub_ids}                 | {stub_assign}          | {stub_release}          | -                       | Pipe support connections |\n"
markdown_output += f"| Intermediate Trans Beams | {intermediate_transverse_ids} | {inter_trans_assign} | {inter_trans_release}   | -                       | Secondary transverse support |\n"
markdown_output += f"| Intermediate Long Beams  | {intermediate_long_ids}    | {inter_long_assign}    | {inter_long_release}    | -                       | Secondary longitudinal support |\n"
markdown_output += f"| Plan Braces              | {plan_brace_ids}           | {plan_brace_assign}    | -                       | {plan_brace_spec}       | Horizontal stability bracing |\n"
markdown_output += f"| Vertical Braces          | {vertical_brace_ids}       | {vertical_brace_assign}| -                       | {vertical_brace_spec}   | Vertical stability bracing |\n"

# Add analysis summary
markdown_output += "\n## Secondary Members Analysis\n\n"
markdown_output += f"- **Bracing Pattern**: {brace_pattern}\n"
markdown_output += f"- **Total Stubs**: {len(stub_ids)} (connecting longitudinal beam pairs)\n"
markdown_output += f"- **Total Intermediate Transverse**: {len(intermediate_transverse_ids)} (from stub midpoints)\n"
markdown_output += f"- **Total Intermediate Longitudinal**: {len(intermediate_long_ids)} (from portal beam midpoints)\n"
markdown_output += f"- **Total Plan Braces**: {len(plan_brace_ids)} (horizontal X-bracing)\n"
markdown_output += f"- **Total Vertical Braces**: {len(vertical_brace_ids)} (vertical V-bracing)\n"

# display(Markdown(markdown_output))

In [ ]:
# Flare support Members
flare_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.Flare):
        line:Line3D
        for line in flare_lines:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    flare_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

flare_long_ids = add_beams(flare_members)
flare_long_assign = assign_profile(flare_long_ids,profile_ids[PiperackMembers.FlareSupportMembers])
flare_long_release = set_start_end_release(flare_long_ids)

# Tree Support Members
tree_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.ElectricalIntrumentation):
        line:Line3D
        for line in tree_support:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    tree_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

tree_ids = add_beams(tree_members)
tree_assign = assign_profile(tree_ids,profile_ids[PiperackMembers.TreeSupportMembers])
tree_release = set_start_end_release(tree_ids)

# Duct Support Members
duct_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.ElectricalIntrumentation):
        line:Line3D
        for line in duct_lines:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    duct_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),
                                             end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

duct_ids = add_beams(duct_members)
duct_assign = assign_profile(duct_ids, profile_ids[PiperackMembers.DuctSupportMembers])
duct_release = set_start_end_release(duct_ids)

# WW Support Members
ww_members = []

for tier_x in tiers:
    if(tier_x.tier_type == TierType.ElectricalIntrumentation):
        line:Line3D
        for line in ww_lines:
            if(line.start.eq_y(tier_x.base)):
                for portal_i in range(1,len(portals)):
                    current_portal = portals[portal_i]
                    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
                    
                    ww_members.append(Beam3D(start=previous_portal.base.shift_y(tier_x.base.y).shift_x(line.start.x),
                                           end=current_portal.base.shift_y(tier_x.base.y).shift_x(line.end.x)))

ww_ids = add_beams(ww_members)
ww_assign = assign_profile(ww_ids, profile_ids[PiperackMembers.WWSupportMembers])
ww_release = set_start_end_release(ww_ids)

# Updated Markdown output
markdown_output = "## Secondary Members Created\n\n"
markdown_output += "| Member Types              | IDs                        | Profile Assignment      | Release Assignment      | Specification           | cláss    |\n"
markdown_output += "| ---                       | ---                        | ---                     | ---                     | ---                     | ---      |\n"
markdown_output += f"| Flare Members            | {flare_long_ids}       | {flare_long_assign}        | {flare_long_release}     | -                      | Flare Support |\n"
markdown_output += f"| Tree Members             | {tree_ids}             | {tree_assign}             | {tree_release}          | -                      | Tree Support |\n"
markdown_output += f"| Duct Members             | {duct_ids}             | {duct_assign}             | {duct_release}          | -                      | Duct Support |\n"
markdown_output += f"| WW Members          | {ww_ids}               | {ww_assign}               | {ww_release}            | -                      | WW Support |\n"

display(Markdown(markdown_output))

## Loads

### Initialization


In [ ]:
from base.load.wind_load import *
add_conc_forces_to_members = add_conc_forces_to_members_fn(load)
add_uniform_forces_to_members = add_uniform_forces_to_members_fn(load)

def beam_list_select_and_display(beam_list):
    select_beams(beam_list.keys())
    display(beam_list)

### Member Import

In [ ]:
nodes = get_node_incidences(geometry=geometry)
beam_objects:dict = get_beam_objects(geometry=geometry,property=None,nodes=nodes)
beam_nos:list = list(beam_objects.keys())

tier_elevations = [tier.base.y for tier in tiers]
portal_zs = [portal.base.z for portal in portals]

### Selfweight

In [ ]:
set_load_case_active(STAAD_objects.load,load_case_no=LoadCase.SelfWeight)

steel_members = filter(lambda beam:beam.start.y >= base_point_of_first_portal.y,beam_objects.values())
concrete_members = filter(lambda beam:beam.start.y < base_point_of_first_portal.y,beam_objects.values())

### Portal Beams And Intermediate Beams

#### Segregation

In [ ]:
# Levels matching with tier levels and z position with portal positions
portal_beams = {}
intermediate_transverse_beams = {}

portal_beam_dict = {portal.base.z:[] for portal in portals}
portal_tier_beams = {portal:{tier:[] for tier in tier_elevations} for portal in portal_zs}

for beam_no,beam in beam_objects.items():
    
    tier_y = None
    portal_z = None
    chosen_one_1,chosen_one_2 = False,False
    
    beam:Beam3D
    
    for tier in tier_elevations:
        if(beam.start.eq_z(beam.end) and beam.start.eq_y(tier) and beam.end.eq_y(tier)):
            chosen_one_1 = True
            tier_y = tier
    
    for portal in portal_zs:
        if(beam.start.eq_z(portal) and beam.end.eq_z(portal)):
            chosen_one_2 = True
            portal_z = portal
    
    if(chosen_one_1 and chosen_one_2):
        portal_beams[beam_no] = beam
        if(tier_y and tier in tier_dict):
            tier_dict[tier_y].add_beam(beam)
        if(portal_z and portal_z in portal_beam_dict):
            portal_beam_dict[portal_z].append(beam)
            
        portal_tier_beams[portal_z][tier_y].append(beam_no)
        
    if(chosen_one_1 and not chosen_one_2):
        intermediate_transverse_beams[beam_no] = beam
        if(tier_y in tier_dict):
            tier_dict[tier_y].add_int_beam(beam)

# beam_list_select_and_display(portal_beams)
# time.sleep(2)
# beam_list_select_and_display(intermediate_transverse_beams)

#### Load Assignment

##### Portal Beams

In [ ]:
for tier_x in tiers:
    for load_x in tier_x.loads:
        set_load_case_active(STAAD_objects.load,load_x.load_case)
        
        if(isinstance(load_x,ConcentratedLoad)):
            add_conc_forces_to_members(tier_x.beams,load_x)

max_portal_to_portal = max([portals[portal_i].base.distance_to(portals[portal_i+1].base) for portal_i in range(len(portals)-1)])

for portal_i in range(len(portals)):
    current_portal = portals[portal_i]
    previous_portal = portals[portal_i - 1] if portal_i > 0 else None
    next_portal = portals[portal_i + 1] if portal_i < len(portals) - 1 else None
    
    dist_1 = previous_portal.base.distance_to(current_portal.base) if previous_portal else max_portal_to_portal
    dist_2 = next_portal.base.distance_to(current_portal.base) if next_portal else max_portal_to_portal
    
    for tier_x in tiers:
        for load_x in tier_x.loads:
            set_load_case_active(STAAD_objects.load,load_x.load_case)
            
            if(isinstance(load_x,UniformLoad)):
                add_uniform_forces_to_members(portal_tier_beams[current_portal.base.z][tier_x.base.y],load_x.factor_force_value(avg([dist_1,dist_2])))

##### Intermediate Transverse Beams

In [ ]:
for tier_x in tiers:
    for load_x in tier_x.clt_loads:
        set_load_case_active(STAAD_objects.load,load_x.load_case)
        
        if(isinstance(load_x,ConcentratedLoad)):
            add_conc_forces_to_members(tier_x.int_beams,load_x)

for tier_x in tiers:
    tier_sorted_z = sorted(tier_x.int_beams,key=lambda beam : beam.start.z)
    for beam in tier_sorted_z:

        closest_portals = sorted(portals,key=lambda portal : portal.base.distance_to(beam.mid()))
        distance = closest_portals[0].base.distance_to(closest_portals[1].base)
        # select_beam(geometry,beam.id)

        for load_x in tier_x.clt_loads:
            set_load_case_active(STAAD_objects.load,load_x.load_case)

            if(isinstance(load_x,UniformLoad)):
                add_uniform_forces_to_members([beam],load_x.factor_force_value(distance))


In [ ]:
# openSTAAD.UpdateStructure()
# beam_list_select_and_display(intermediate_transverse_beams)

### Columns

#### Column Identification

In [ ]:
portal_z_set = set(portal_zs)

columns_nos = [beam_no for beam_no,beam in beam_objects.items() if beam.start.x == beam.end.x and beam.start.z == beam.end.z and  beam.start.z in portal_z_set and abs(beam.start.y - beam.end.y) > 0.1]
x_coordinates = sorted(list(set([beam_objects[col_i].start.x for col_i in columns_nos])))
z_coordinates = sorted(list(set([beam_objects[col_i].start.z for col_i in columns_nos])))

columns_x = {x:[] for x in x_coordinates}
columns_z = {z:[] for z in z_coordinates}

for col_i in columns_nos:
    beam = beam_objects[col_i]
    columns_x[beam.start.x].append(beam)

for col_i in columns_nos:
    beam = beam_objects[col_i]
    columns_z[beam.start.z].append(beam)

#### Wind Load Definition

In [ ]:
ngl = -0.5

add_ngl = add_elevation_to_wind_load_list_fn(ngl)

transverse_wind_loads = [
    WindLoad(start_ele=0.0, end_ele=10.0, windward=0.5408, leeward=0.4501),
    WindLoad(start_ele=10.0, end_ele=15.0, windward=0.5962, leeward=0.4962),
    WindLoad(start_ele=15.0, end_ele=20.0, windward=0.6191, leeward=0.5153),
    WindLoad(start_ele=20.0, end_ele=25.0, windward=0.6484, leeward=0.5396),
    WindLoad(start_ele=25.0, end_ele=30.0, windward=0.6783, leeward=0.5645)
]

longitudinal_wind_loads = [
    WindLoad(start_ele=0.0, end_ele=10.0, windward=0.2704, leeward=0.2279),
    WindLoad(start_ele=10.0, end_ele=15.0, windward=0.2981, leeward=0.2513),
    WindLoad(start_ele=15.0, end_ele=20.0, windward=0.3096, leeward=0.2610),
    WindLoad(start_ele=20.0, end_ele=25.0, windward=0.3242, leeward=0.2733),
    WindLoad(start_ele=25.0, end_ele=30.0, windward=0.3392, leeward=0.2859)
]

add_ngl(transverse_wind_loads)
add_ngl(longitudinal_wind_loads)

display(Markdown(wind_load_definition_markdown(transverse_wind_loads)))
display(Markdown(wind_load_definition_markdown(longitudinal_wind_loads)))

#### Wind Load Assignment

##### Transverse Direction

In [ ]:
generated_loads = []
set_load_case_active(load,LoadCase.WindColumn_GX)
pedestal_offset_start_y = base_point_of_first_portal.y

for z in x_coordinates:
    for col in columns_x[z]:
        for wind_l in transverse_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)

                force = wind_l.windward if z==x_coordinates[0] else wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX))

set_load_case_active(load,LoadCase.WindColumn_GX_Opposite)
for z in x_coordinates:
    for col in columns_x[z]:
        for wind_l in transverse_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                
                force = -wind_l.windward if z==x_coordinates[-1] else -wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX_Opposite)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GX,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GX_Opposite))

# display(Markdown(wind_load_assignment_markdown('Wind GX Forces on Columns',generated_loads)))

##### Longitudinal Direction

In [ ]:
generated_loads = []
set_load_case_active(load,LoadCase.WindColumn_GZ)

for z in z_coordinates:
    for col in columns_z[z]:
        for wind_l in longitudinal_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)

                force = wind_l.windward if z==z_coordinates[0] else wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ))

set_load_case_active(load,LoadCase.WindColumn_GZ_Opposite)

for z in z_coordinates:
    for col in columns_z[z]:
        for wind_l in longitudinal_wind_loads:
            d1 = max([col.start.y,wind_l.start_ele])
            d2 = min([col.end.y,wind_l.end_ele])

            if(col.start.y<=d1<=col.end.y and col.start.y<=d2<=col.end.y):
                d1 = d1 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                d2 = d2 - col.start.y - (foundation_depth if col.end.y == pedestal_offset_start_y else 0)
                
                force = -wind_l.windward if z==z_coordinates[-1] else -wind_l.leeward
                generated_loads.append((col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ_Opposite)))
                add_member_uniform_force(load,col.id,UniformLoad(MemberDirection.GZ,force,d1_value=d1,d2_value=d2,load_case=LoadCase.WindColumn_GZ_Opposite))

# display(Markdown(wind_load_assignment_markdown('Wind GZ Forces on Columns',generated_loads)))

### Post Build Script

In [ ]:
# openSTAAD.UpdateStructure()

### Member Selector

In [ ]:
selected_members = get_selected_beam_nos(geometry)
copy(f'members={selected_members},')

display(Markdown(f'copied **{len(selected_members)}** members : {selected_members}'))